In [1]:
#importing required packages

import requests
import scrapy
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
import string
import pandas as pd

In [2]:
#url of the website to scrape

url='https://internshala.com/fresher-jobs'

In [3]:
#raw html data from website

html=requests.get(url).content
#html

In [4]:
#html in well formatted form

pretty_html=BeautifulSoup(html).prettify()
#pretty_html

In [5]:
#creating scrapy's selector object to select the html above

sel=Selector(text=pretty_html)

In [6]:
#final, the dictionary which would be later converted to a dataframe, final_df, to store the scraped data

final={'ID':[],'Job_Post':[],'Company':[],'Location':[],'CTC':[]}
final_df=pd.DataFrame()

In [7]:
#scraping available job's ID

final['ID']=sel.xpath('//div[@class="container-fluid individual_internship visibilityTrackerItem"]/@internshipid').extract()
final['ID']

['1352353',
 '1349122',
 '1349016',
 '1349799',
 '1350496',
 '1350355',
 '1339564',
 '1349846',
 '1348418',
 '1348595',
 '1348054',
 '1343650',
 '1348373',
 '1348235',
 '1347553',
 '1347639',
 '1345697',
 '1347504',
 '1347272',
 '1347212',
 '1345866',
 '1346312',
 '1342210',
 '1343488',
 '1331731',
 '1345400',
 '1345945',
 '1345589',
 '1345189',
 '1345281',
 '1344273',
 '1343726',
 '1343707',
 '1342471',
 '1342358',
 '1341248',
 '1341491',
 '1341824',
 '1341072',
 '1341477']

In [8]:
#scraping available job's name

final['Job_Post']=sel.xpath('//div[@class="heading_4_5 profile"]/a').xpath('normalize-space(./text())').extract()
final['Job_Post']

['Marketing Executive',
 'Embedded Systems Associate',
 'Product Developer - Science',
 'Salesforce Developer',
 'Product Engineer Trainee',
 'Outreach Associate',
 'Corporate Sales Specialist',
 'Junior Social Media Marketing Manager',
 'Full Stack Developer',
 'Full Stack Software Engineer',
 'iOS App Developer',
 'Junior Industrial And Production Engineer',
 'Accountant',
 'Business Development Manager',
 'Sales Executive',
 'Admission Counselor (Sales)',
 'Sales Executive',
 'Business Development Executive (Inside Sales)',
 'Content Writer',
 'Associate Front End Developer - Shopify',
 'Operations Executive',
 'Content Writer',
 'Django/Full Stack Developer',
 'Operations & Business Development Associate',
 'Associate Software Developer',
 'Associate Full Stack Developer',
 'UI/UX Designer',
 'Business Marketing Manager',
 'Android App Developer',
 'Assistant Manager - Sales & Marketing',
 'Associate UI/UX Designer',
 'Business Development Executive',
 'Full Stack Developer',
 'Bus

In [9]:
final['Company']=sel.xpath('//a[@class="link_display_like_text"]').xpath('normalize-space(./text())').extract()
final['Company']

['Lotrec AsiaPacific Private Limited',
 'Sigsenz Technologies',
 'Open Door Education',
 'Aekot Consulting And Technologies Private Limited',
 'Wisestep',
 'Formskart',
 'Harbinger Foods Private Limited',
 'Muskurahat Foundation',
 'Beyond 8',
 'Flair Labs',
 'WebMOBI',
 'Würfel Kuche',
 'Shubh Fashion',
 'Build With Innovation Private Limited',
 'FlippAR',
 'Sky Education Group',
 'Sheth Info',
 'GREedge',
 'How I Got My Job',
 'Seventh Triangle Consulting',
 'SysCloud Technologies Private Limited',
 'Writeronrent',
 'TutorBin',
 'MiM-Essay',
 'Hyperface Technologies Private Limited',
 'WebMOBI',
 'Assist 2 Path Tech',
 'MentorBoxx',
 'Lattice Innovations Private Limited',
 'S3 Infosoft',
 'LearnMall',
 'Agrawal Goyal & Jalan',
 'Analystt.ai',
 'Finwego',
 'Crescibit Technology Private Limited',
 'BRANDanswers',
 'Ayasya Digital',
 'Zell Education',
 'Boka Tech OPC Private Limited',
 'Snazzy Align']

In [10]:
#scraping the city name for the job (the toughest part, due to the fact that a single job post can have mutiple job locations)

#scraping each job's locations in a different list thus forming a nested list,ind_location
ind_location=[]
final['Location']=[]
for i in final['ID']:
    ind_location.append(sel.xpath("//div[@internshipid='" + i + "']//a[@class='location_link']").xpath('normalize-space(./text())').extract())

#converting the nested list above and storing in a normal list, final['Location']  
for i in ind_location:
    s=''
    c=0
    if len(i)>1:
        for j in i:
            if c>0:
                s=s+', '+j
            else:
                s=s+j
            c+=1
    else:
        s=i[0]
    final['Location'].append(s)

final['Location']

['Mumbai',
 'Chennai, Pune, Bangalore, Hyderabad, Mumbai',
 'Bangalore',
 'Bangalore',
 'Delhi, Kolkata, Pune, Hyderabad, Mumbai, Jaipur, Bangalore',
 'Ahmedabad, Chennai, Delhi, Guwahati, Indore, Kolkata, Lucknow, Pune, Ranchi, Secunderabad, Hyderabad, Kochi, Howrah, Jaipur',
 'Gurgaon',
 'Mumbai',
 'Remote',
 'Mumbai',
 'Remote',
 'Bangalore',
 'Kolkata',
 'Delhi',
 'Bangalore',
 'Delhi, Ghaziabad, Noida',
 'Ahmedabad',
 'Remote',
 'Remote',
 'Remote',
 'Remote',
 'Remote',
 'Gurgaon',
 'Delhi',
 'Bangalore',
 'Remote',
 'Delhi, Kolkata, Jaipur, Kanpur Dehat, Jammu, Kerela, Mali (Mali)',
 'Delhi, Pune, Mumbai',
 'Remote',
 'Pune',
 'Bangalore',
 'Jaipur',
 'Remote',
 'Remote',
 'Mumbai',
 'Remote',
 'Faridabad, Delhi, Gurgaon, Noida',
 'Mumbai',
 'Remote',
 'Bangalore']

In [11]:
to_remove={0:["\n                ","\n               "],\
           1:['\n             ','\n            ']}

def remove_(attr,remove):
    for i in remove:
        final[attr]=[f.replace(i, "") for f in final[attr]]

final['CTC']=sel.xpath('//div[@class="internship_other_details_container"]/div[1]/div[2]/div[2]/text()').extract()                                   
remove_('CTC',to_remove[0])
final['CTC'] = ['₹'+v for i, v in enumerate(final['CTC']) if(i % 2)]
final['CTC']

['₹ 3 - 3.2 LPA',
 '₹ 3 - 4 LPA',
 '₹ 5 - 7 LPA',
 '₹ 3.25 LPA',
 '₹ 5.5 - 7 LPA',
 '₹ 3 - 3.1 LPA',
 '₹ 3 - 3.25 LPA',
 '₹ 3 - 4 LPA',
 '₹ 3 - 3.5 LPA',
 '₹ 3 - 4 LPA',
 '₹ 3 LPA',
 '₹ 3 - 3.1 LPA',
 '₹ 3 LPA',
 '₹ 3 - 5 LPA',
 '₹ 3 LPA',
 '₹ 3 LPA',
 '₹ 3 - 3.2 LPA',
 '₹ 3.75 LPA',
 '₹ 3 - 3.5 LPA',
 '₹ 3.2 - 3.6 LPA',
 '₹ 3 - 3.01 LPA',
 '₹ 3 - 3.5 LPA',
 '₹ 5 - 8 LPA',
 '₹ 4 - 5 LPA',
 '₹ 6 - 7 LPA',
 '₹ 3 LPA',
 '₹ 3 - 3.5 LPA',
 '₹ 4 - 8.5 LPA',
 '₹ 3 LPA',
 '₹ 3 - 3.6 LPA',
 '₹ 3 - 3.1 LPA',
 '₹ 3 LPA',
 '₹ 3 - 4 LPA',
 '₹ 3.4 - 4.6 LPA',
 '₹ 3.5 - 5 LPA',
 '₹ 4 - 6 LPA',
 '₹ 3 - 3.6 LPA',
 '₹ 3 - 3.1 LPA',
 '₹ 3 - 7 LPA',
 '₹ 5 - 10 LPA']

In [12]:
final['Apply By']=sel.xpath('//div[@class="internship_other_details_container"]/div[@class="other_detail_item_row"][2]//div[@class="item_body"]').xpath('normalize-space(./text())').extract()
final['Apply By']

["8 Aug' 21",
 "8 Aug' 21",
 "7 Aug' 21",
 "7 Aug' 21",
 "7 Aug' 21",
 "7 Aug' 21",
 "7 Aug' 21",
 "7 Aug' 21",
 "6 Aug' 21",
 "6 Aug' 21",
 "6 Aug' 21",
 "6 Aug' 21",
 "6 Aug' 21",
 "6 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "5 Aug' 21",
 "4 Aug' 21",
 "4 Aug' 21",
 "8 Aug' 21",
 "4 Aug' 21",
 "2 Aug' 21",
 "2 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21",
 "1 Aug' 21"]

In [13]:
final_df=pd.DataFrame(final)
final_df

,ID,Job_Post,Company,Location,CTC,Apply By
0,1352353,Marketing Executive,Lotrec AsiaPacific Private Limited,Mumbai,₹ 3 - 3.2 LPA,8 Aug' 21
1,1349122,Embedded Systems Associate,Sigsenz Technologies,"Chennai, Pune, Bangalore, Hyderabad, Mumbai",₹ 3 - 4 LPA,8 Aug' 21
2,1349016,Product Developer - Science,Open Door Education,Bangalore,₹ 5 - 7 LPA,7 Aug' 21
3,1349799,Salesforce Developer,Aekot Consulting And Technologies Private Limited,Bangalore,₹ 3.25 LPA,7 Aug' 21
4,1350496,Product Engineer Trainee,Wisestep,"Delhi, Kolkata, Pune, Hyderabad, Mumbai, Jaipu...",₹ 5.5 - 7 LPA,7 Aug' 21
5,1350355,Outreach Associate,Formskart,"Ahmedabad, Chennai, Delhi, Guwahati, Indore, K...",₹ 3 - 3.1 LPA,7 Aug' 21
6,1339564,Corporate Sales Specialist,Harbinger Foods Private Limited,Gurgaon,₹ 3 - 3.25 LPA,7 Aug' 21
7,1349846,Junior Social Media Marketing Manager,Muskurahat Foundation,Mumbai,₹ 3 - 4 LPA,7 Aug' 21
8,1348418,Full Stack Developer,Beyond 8,Remote,₹ 3 - 3.5 LPA,6 Aug' 21
9,1348595,Full Stack Software Engineer,Flair Labs,Mumbai,₹ 3 - 4 LPA,6 Aug' 21
